# Other solutions

In questo notebook si effettua il confronto fra le varie soluzioni che sono state sperimentate prima di concludere che l'SC3 fosse il migliore.

## Dependencies

In [1]:
import pandas as pd
from haversine import haversine
from utils.utils import get_config_from_yaml, load_data, filter_by_level
from utils.simulation import Simulation
from utils.distances import slender_distance
from routing.googlerouting import GoogleRouting
from routing.geneticalgorithmgraph import GAGraph
from clustering.adjustedkmeans import AdjustedKMeans

## Configuration loading

In [2]:
YAML = "configs/example.yaml"

config = get_config_from_yaml(YAML)
depot = [config.depot.latitude, config.depot.longitude]
start_date = pd.to_datetime(config.data.period.start_date)
end_date = pd.to_datetime(config.data.period.end_date)

## Data loading and preprocessing

In [3]:
data = load_data(config.data.file_path)
data = data[config.data.period.start_date: config.data.period.end_date]

## Test

In [4]:
def run_simulation(config, name):
    simulation = Simulation(depot, config, window_size=6, max_size=200,
                            filter_function=filter_by_level, filter_kwargs={})
    routes = simulation.compute_simulation(data, pd.to_datetime(start_date), pd.to_datetime(end_date), speed=30, emp_time=60, debug=False)
    simulation.print_simulation()
    simulation.to_csv(f'output/{name}.csv')


### Baseline: Standard k-Means + Christofides

In [5]:
kmeans_kwargs = {
    'max_size' : 200,
    'balanced': False,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 0, 1),
}

routing_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 1,
}

baseline_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': kmeans_kwargs,
  'graph_class': GoogleRouting,
  'graph_kwargs': routing_kwargs,
}

run_simulation(baseline_config, 'Baseline')

# Simulation #
Shifts executed: 120.
Total distance: 19222.724620000008 km.
Bins emptied: 130845.
Vehicles involved: 709.


### Google Routing

In [6]:
GR_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 20,
}

GR_config = {
  'cluster_class': None,
  'cluster_kwargs': {},
  'graph_class': GoogleRouting,
  'graph_kwargs': GR_kwargs,
}

run_simulation(GR_config, 'GoogleRouting')

# Simulation #
Shifts executed: 120.
Total distance: 18661.584350000012 km.
Bins emptied: 130845.
Vehicles involved: 709.


### SC2G - Shape-Controlled Clustering + Genetic algorithm

In [7]:
clustering_kwargs = {
    'max_size' : 200,
    'balanced': True,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 1, 0),
}

# GA settings
pop_size = 500
elite_size = int(0.05*pop_size)
mutation_rate = 0.1
generations = 200

GA_kwargs = {
    'pop_size': pop_size,
    'elite_size': elite_size, 
    'mutation_rate': mutation_rate,
    'generations': generations,
    'metric': 'km',
}

SC2G_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': clustering_kwargs,
  'graph_class': GAGraph,
  'graph_kwargs': GA_kwargs,
}

run_simulation(SC2G_config, 'SC2G')

# Simulation #
Shifts executed: 120.
Total distance: 18706.83744 km.
Bins emptied: 130845.
Vehicles involved: 709.


### SC3 bilanciato

In [8]:
clustering_balanced = {
    'max_size' : 200,
    'balanced': True,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 1, 0),
}

routing_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 1,
}

SC3_balanced_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': clustering_kwargs,
  'graph_class': GoogleRouting,
  'graph_kwargs': routing_kwargs,
}

run_simulation(SC3_balanced_config, 'SC3Balanced')

# Simulation #
Shifts executed: 120.
Total distance: 17790.80992 km.
Bins emptied: 130845.
Vehicles involved: 709.
